In [1]:
%matplotlib inline
import torch
import torch.nn.functional as F
from torch.optim.optimizer import Optimizer
import math
import random
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import tqdm
from matplotlib import pyplot as plt
from copy import deepcopy
import os
import datetime
import pickle
import copy
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
device = torch.device("cuda:0")
# device = torch.device("cpu")

In [3]:
batch_size = 512
plot_num = 50000
epochs = 30

In [ ]:
# this part is different form POEM
class SessionData(object):
    def __init__(self,session_index,session_id,items_indexes):
        self.session_index = session_index
        self.session_id = session_id
        self.item_list = items_indexes
    def generate_seq_datas(self,session_length,padding_idx=0,predict_length=1):
        sessions = []
        if len(self.item_list)<2:
            self.item_list.append[self.item_list[0]]
        if predict_length==1:
#             # when session length>=3
#             for i in range(1,len(self.item_list)-1):
            # when session length >=2
            for i in range(len(self.item_list)-1):
                if i <session_length:
                    train_data = [0 for _ in range(session_length-i-1)]
                    train_data.extend(self.item_list[:i+1])
                    train_data.append(self.item_list[i+1])
                else:
                    train_data = self.item_list[i+1-session_length:i+1]
                    train_data.append(self.item_list[i+1])
                sessions.append(train_data)
        else:
            # To be continue if necessary
            pass
        return self.session_index,sessions
    def __str__(self):
        info = " session index = {}\n session id = {} \n the length of item list= {} \n the fisrt item index in item list is {}".format(self.session_index,self.session_id,len(self.item_list),self.item_list[0])
        return info

In [ ]:
class SessionDataSet(object):
    def __init__(self,train_file,test_file,padding_idx=0):
        super(SessionDataSet,self).__init__()
        self.index_count = 0
        self.session_count = 0
        self.train_count = 0
        self.test_count = 0
        self.max_session_length = 0

        self.padding_idx = padding_idx
        self.item2index = dict()
        self.index2item = dict()
        self.session2index = dict()
        self.index2session = dict()
        self.item_total_num = dict()
        self.item2index["<pad>"] = padding_idx
        self.index2item[padding_idx] = "<pad>"
        self.train_data = self.load_data(train_file)
        print("training set is loaded, # index: ",len(self.item2index.keys()))
        self.train_count = self.session_count
        print("train_session_num",self.train_count)
        self.test_data = self.load_data(test_file)
        print("testing set is loaded, # index: ",len(self.index2item.keys()))
        print("# item",self.index_count)
        self.test_count = self.session_count-self.train_count
        print("# test session:",self.test_count)
        self.all_training_data = []
        self.all_testing_data = []
        self.all_meta_training_data = []
        self.all_meta_testing_data = []
        self.train_session_length = 0
        self.test_session_length = 0
    
    def load_data(self,file_path):
        data =  pickle.load(open(file_path, 'rb'))
        session_ids = data[0]
        session_data = data[1]
        session_label = data[2]

        result_data = []
        lenth = len(session_ids)
        print("# session",lenth)

        last_session_id = session_ids[0]
        
        session_item_indexes = []

        for item_id in session_data[0]:
            if item_id not in self.item2index.keys():
                self.index_count+=1
                self.item2index[item_id] = self.index_count
                self.index2item[self.index_count] = item_id
                self.item_total_num[self.index_count] = 0
            session_item_indexes.append(self.item2index[item_id])
            self.item_total_num[self.item2index[item_id]] += 1
        target_item = session_label[0]
        if target_item not in self.item2index.keys():
            self.index_count+=1
            self.item2index[target_item] = self.index_count
            self.index2item[self.index_count] = target_item
            self.item_total_num[self.index_count] = 0
        session_item_indexes.append(self.item2index[target_item])
        self.item_total_num[self.item2index[target_item]] += 1

        for session_id,items,target_item in zip(session_ids,session_data,session_label):
            if session_id!=last_session_id:

                self.session_count+=1
                self.session2index[last_session_id] = self.session_count
                self.index2session[self.session_count] = last_session_id
                if len(session_item_indexes)>self.max_session_length:
                    self.max_session_length = len(session_item_indexes)
                new_session = SessionData(self.session_count,last_session_id,session_item_indexes)
                result_data.append(new_session)
                last_session_id = session_id
                session_item_indexes = []
                for item_id in items:
                    if item_id not in self.item2index.keys():
                        self.index_count+=1
                        self.item2index[item_id] = self.index_count
                        self.index2item[self.index_count] = item_id
                        self.item_total_num[self.index_count] = 0
                    session_item_indexes.append(self.item2index[item_id])
                    self.item_total_num[self.item2index[item_id]] += 1
                if target_item not in self.item2index.keys():
                    self.index_count+=1
                    self.item2index[target_item] = self.index_count
                    self.index2item[self.index_count] = target_item
                    self.item_total_num[self.index_count] = 0
                session_item_indexes.append(self.item2index[target_item])
                self.item_total_num[self.item2index[target_item]] += 1
            else:
                continue

        self.session_count+=1
        self.session2index[last_session_id] = self.session_count
        new_session = SessionData(self.session_count,last_session_id,session_item_indexes)
        result_data.append(new_session)
        print("loaded")
        print(new_session)
        
        return result_data
    

    def get_batch(self,batch_size,session_length=10,predict_length=1,all_data=None,phase="train",neg_num=1,sampling_mathod="random"):

        if phase == "train":
            if all_data is None:
                all_data = self.get_all_training_data(session_length)
            indexes = np.random.permutation(all_data.shape[0])
            all_data = all_data[indexes]
        else:
            if all_data is None:
                all_data = self.get_all_testing_data(session_length)
        
        sindex = 0
        eindex = batch_size
        while eindex < all_data.shape[0]:
            batch = all_data[sindex: eindex]

            temp = eindex
            eindex = eindex + batch_size
            sindex = temp
            if phase =="train":
                batch = self.divid_and_extend_negative_samples(batch,session_length=session_length,predict_length=predict_length,neg_num=neg_num,method=sampling_mathod)
            else:
                batch = [batch[:,:session_length],batch[:,session_length:]]
            yield batch

        if eindex >= all_data.shape[0]:
            batch = all_data[sindex:]
            if phase =="train":
                batch = self.divid_and_extend_negative_samples(batch,session_length=session_length,predict_length=predict_length,neg_num=neg_num,method=sampling_mathod)
            else:
                batch = [batch[:,:session_length],batch[:,session_length:]]
            yield batch
    
    def divid_and_extend_negative_samples(self,batch_data,session_length,predict_length=1,neg_num=1,method="random"):
        """
        divid and extend negative samples
        """
        neg_items = []
        if method == "random":
            for session_and_target in batch_data:
                neg_item = []
                for i in range(neg_num):
                    rand_item = random.randint(1,self.index_count)
                    while rand_item in session_and_target or rand_item in neg_item:
                        rand_item = random.randint(1,self.index_count)
                    neg_item.append(rand_item)
                neg_items.append(neg_item)
        else:

            total_list = set()
            for session in batch_data:
                for i in session:
                    total_list.add(i) 
            total_list = list(total_list)
            total_list =  sorted(total_list, key=lambda item: self.item_total_num[item],reverse=True)
            for i,session in enumerate(batch_data):
                np.random.choice(total_list)
        session_items = batch_data[:,:session_length]
        target_item = batch_data[:,session_length:]
        neg_items = np.array(neg_items)
        return [session_items,target_item,neg_items]
    
    def get_all_training_data(self,session_length,predict_length=1):
        if len(self.all_training_data)!=0 and self.train_session_length==session_length:
#             print("The build is complete and there is no need to repeat the build")
            return self.all_training_data
        print("Start building the all training dataset")
        all_sessions = []
        for session_data in self.train_data:
            # 前session_length为session，后predict_length为target_item
            session_index,sessions = session_data.generate_seq_datas(session_length,padding_idx=self.padding_idx)
            if sessions is not None:
                all_sessions.extend(sessions)
        all_sessions = np.array(all_sessions)
        self.all_training_data = all_sessions
        self.train_session_length=session_length
        print("The total number of training samples is：",all_sessions.shape)
        return all_sessions
    
    def get_all_testing_data(self,session_length,predict_length=1):
        if len(self.all_testing_data)!=0 and self.test_session_length==session_length:
            return self.all_testing_data
        all_sessions = []
        for session_data in self.test_data:
            session_index,sessions = session_data.generate_seq_datas(session_length,padding_idx=self.padding_idx)
            if sessions is not None:
                all_sessions.extend(sessions)
        all_sessions = np.array(all_sessions)
        self.all_testing_data = all_sessions
        self.test_session_length=session_length
        print("The total number of testing samples is：",all_sessions.shape)
        return all_sessions

    def __getitem__(self,idx):
        pass
    
    def __len__(self):
        pass

In [6]:
# dataset = SessionDataSet(train_file="../data/retailrocket/train.txt",test_file="../data/srgnn/retailrocket/test.txt")
# dataset = SessionDataSet(train_file="../data/diginetica/train.txt",test_file="../data/srgnn/diginetica/test.txt")
dataset = SessionDataSet(train_file="../data/yoochoose1_4/train.txt",test_file="../data/srgnn/yoochoose1_4/test.txt")
# dataset = SessionDataSet(train_file="../data/yoochoose1_64/train.txt",test_file="../data/srgnn/yoochoose1_64/test.txt")

# session 369859
loaded
 session index = 116167
 session id = 11497318 
 the length of item list= 3 
 the fisrt item index in item list is 373
training set is loaded, # index:  17377
train_session_num 116167
# session 55898
loaded
 session index = 131491
 session id = 11560908 
 the length of item list= 3 
 the fisrt item index in item list is 4799
testing set is loaded, # index:  17746
# item 17745
# test session: 15324


In [7]:
def bpr_loss(r):
    return torch.sum(-torch.log(torch.sigmoid(r)))
def get_hit_num(pred,y_truth):
    """
        pred: numpy type(batch_size,k) 
        y_truth: list type (batch_size,groudtruth_num)
    """

    hit_num = 0
    for i in range(len(y_truth)):
        for value in y_truth[i]:
            hit_num += np.sum(pred[i]==value)
    return hit_num

def get_rr(pred,y_truth):
    rr=0.
    for i in range(len(y_truth)):
        for value in y_truth[i]:
            hit_indexes = np.where(pred[i]==value)[0]
            for hit_index in hit_indexes:
                rr += 1/(hit_index+1)
    return rr

def get_dcg(pred,y_truth):
    y_pred_score = np.zeros_like(pred)

    for i in range(len(y_truth)):

        for j,y_pred in enumerate(pred[i]):
            if y_pred == y_truth[i][0]:
                y_pred_score[i][j]=1
    gain = 2 ** y_pred_score - 1
    discounts = np.tile(np.log2(np.arange(pred.shape[1]) + 2),(len(y_truth),1))
    dcg = np.sum(gain / discounts,axis=1)
    return dcg

def get_ndcg(pred,y_truth):
    dcg = get_dcg(pred, y_truth)
    idcg = get_dcg(np.concatenate((y_truth,np.zeros_like(pred)[:,:-1]-1),axis=1), y_truth)
    ndcg = np.sum(dcg / idcg)

    return ndcg

def dcg_score(y_pre, y_true, k):
    y_pre_score = np.zeros(k)
    if len(y_pre) > k:
        y_pre = y_pre[:k]
    for i in range(len(y_pre)):
        pre_tag = y_pre[i]
        if pre_tag in y_true:
            y_pre_score[i] = 1
    gain = 2 ** y_pre_score - 1
    discounts = np.log2(np.arange(k) + 2)
    return np.sum(gain / discounts)


def ndcg_score(y_pre, y_true, k=5):
    dcg = dcg_score(y_pre, y_true, k)
    idcg = dcg_score(y_true, y_true, k)
    return dcg / idcg

loss_function = torch.nn.CrossEntropyLoss()

In [8]:
# SelfAttention Layer
class SelfAttention(torch.nn.Module):
    def __init__(self, method, hidden_size,activate="selu",dropout=0):
        super(SelfAttention, self).__init__()
        self.config = list()
        self.method = method
        self.hidden_size = hidden_size
        if self.method not in ['dot', 'general']:
            raise ValueError(self.method, "Attention method do not exists.")

        if self.method == "dot":
            self.query = torch.nn.Linear(self.hidden_size *2, self.hidden_size*2)
            self.key = torch.nn.Linear(self.hidden_size*2, self.hidden_size*2)
            torch.nn.init.constant_(self.query.bias,0)
            torch.nn.init.constant_(self.key.bias,0)

        if self.method == "general":
            self.attention = torch.nn.Linear(self.hidden_size*2, self.hidden_size*2)
            torch.nn.init.constant_(self.attention.bias,0)
        
        if activate == "relu":
            self.activate = torch.relu
        elif activate == "tanh":
            self.activate = torch.tanh
        elif activate == "elu":
            self.activate = torch.nn.ELU()
        elif activate == "selu":
            self.activate = torch.selu
        else:
            self.activate = torch.sigmoid
        
        self.dropout = torch.nn.Dropout(dropout)
        torch.nn.utils.clip_grad_norm_(self.parameters(),max_norm=110)

    def dot_score(self, encoder_output,is_train=True,weights=None):

        if weights is None:
            if is_train:
                query = self.dropout(self.activate(self.query(encoder_output)))
                key = self.dropout(self.activate(self.key(encoder_output)))
            else:
                query = self.activate(self.query(encoder_output))
                key = self.activate(self.key(encoder_output))
        else:
            query = self.activate(torch.matmul(encoder_output,weights[0].t())+weights[1])
            key = self.activate(torch.matmul(encoder_output,weights[2].t())+weights[3])
        dot = query.bmm(key.permute(0, 2, 1))
        return dot

    def general_score(self, encoder_output,is_train=True,weights=None):
        if weights is None:
            if is_train:
                energy = self.dropout(self.activate(self.attention(encoder_output)))
            else:
                energy = self.activate(self.attention(encoder_output))
        else:
            energy = self.activate(torch.matmul(encoder_output,weights[0].t())+weights[1])
        return encoder_output.bmm(energy.permute(0, 2, 1))

    def forward(self, encoder_outputs, mask=None,is_train=True):
        # (batch_size,length,dim)
        if self.method == "general":
            attention_energies = self.general_score(encoder_outputs,is_train=is_train)
        elif self.method == "dot":
            attention_energies = self.dot_score(encoder_outputs,is_train=is_train)

        #  (batch_size,length,length)
        attention_energies.div_(torch.sqrt(torch.tensor(self.hidden_size, dtype=torch.float)))
        if mask is not None:
            new_mask = (1 - (1 - mask.float()).unsqueeze(1).permute(0, 2, 1).bmm(
                (1 - mask.float()).unsqueeze(1)))

            attention_energies = attention_energies - new_mask*1e12
            weights = F.softmax(attention_energies, dim=2)
            weights = weights*(1-new_mask)
            # batch_size,length,length)*(batch_size,length,dim)->(batch_size,length,dim)->(batch_size,1,dim)->(batch_size,dim)
            outputs = weights.bmm(encoder_outputs)
            outputs.div_(mask.shape[1]-torch.sum(mask,dim=1).unsqueeze(1).unsqueeze(2).repeat((1,mask.shape[1],outputs.shape[2])).float())
            outputs = outputs.sum(dim=1).squeeze(1)
        else:
            weights = F.softmax(attention_energies, dim=2)
            # (batch_size,length,length)*(batch_size,length,dim)->(batch_size,length,dim)->(batch_size,1,dim)->(batch_size,dim)
            outputs = (weights.bmm(encoder_outputs).sum(dim=1) / encoder_outputs.shape[1]).squeeze(1)
        sa_weights = weights.sum(dim=1).squeeze(1)
        return outputs, sa_weights
    

In [ ]:
class POEM(torch.nn.Module):
    def __init__(self, hidden_size=64, itemNum=0, posNum=0, padding_idx=0, dropout=0.5,attention_method="dot",head_num=4,
                 activate="selu",session_length=20):
        super(POEM, self).__init__()
        self.padding_idx = padding_idx
        self.hidden_size = hidden_size
        self.head_num = head_num
        self.session_length = session_length
        if activate == "sigmoid":
            self.activate = torch.sigmoid
        elif activate == "tanh":
            self.activate = torch.tanh
        elif activate == "relu":
            self.activate = torch.relu
        elif activate == "elu":
            self.activate = torch.nn.ELU()
        else:
            self.activate = torch.selu
        self.dropout = torch.nn.Dropout(dropout)
        
        self.item_embedding = torch.nn.Embedding(itemNum, hidden_size, padding_idx=self.padding_idx,max_norm=1.5)
        
        self.position_embedding = torch.nn.Embedding(posNum,hidden_size,padding_idx=self.padding_idx,max_norm=1.5)
    
        self.position_weights = torch.nn.Embedding(posNum,1,padding_idx=self.padding_idx)
        
        self.self_attention = SelfAttention(attention_method, hidden_size,activate=activate,dropout=dropout).to(device)
        torch.nn.init.constant_(self.item_embedding.weight[0],0)
        torch.nn.init.constant_(self.position_embedding.weight[0],0)
        torch.nn.init.constant_(self.position_weights.weight,1)
        torch.nn.init.constant_(self.position_weights.weight[0],0)
        
        self.gen_mlp = torch.nn.Linear(hidden_size*2, hidden_size)
        self.cur_mlp = torch.nn.Linear(hidden_size, hidden_size)
        self.deep_mlp = torch.nn.Linear(hidden_size*3, hidden_size,bias=False)
        
        self.layer_norm1 = torch.nn.LayerNorm(hidden_size*2)
        self.layer_norm2 = torch.nn.LayerNorm(hidden_size)
        
    def forward(self, session,item=None,bpr_loss=False,neg_num=50):

        mask = (session!=0).float()
        length = torch.sum(mask,1).unsqueeze(1).expand((session.shape[0],self.hidden_size))
        mask = mask.unsqueeze(2).expand((session.shape[0],session.shape[1],self.hidden_size))
        session_item_embeddings = self.item_embedding(session) * mask
        positions = session.shape[1] - torch.arange(0,session.shape[1]).unsqueeze(0).expand_as(session).to(device)
        session_position_embeddings = self.dropout(self.position_embedding(positions))*mask
        session_item_vecs = torch.cat((session_item_embeddings,session_position_embeddings), dim=2)
        attention_mask = (session == self.padding_idx)
        sa_output, sa_weights = self.self_attention(session_item_vecs, attention_mask)
        sa_output = self.layer_norm1(sa_output)

        session_position_weights = self.dropout(self.position_weights(positions))*mask
        sa_weights = sa_weights.unsqueeze(2).expand_as(session_item_embeddings)
        session_item_vecs2 = session_item_embeddings * session_position_weights * sa_weights
        psa_output = torch.sum(session_item_vecs2, dim=1)/length
        psa_output = self.layer_norm2(psa_output)

        gen_output = self.dropout(self.activate(self.gen_mlp(sa_output)))
        cur_output = self.dropout(self.activate(self.cur_mlp(session_item_embeddings[:,-1])))
        deep_output = self.dropout(self.activate(self.deep_mlp(torch.cat((sa_output,session_item_embeddings[:,-1]),1))))
        
        session_output = self.layer_norm2(gen_output* cur_output+deep_output +psa_output)
        
        result = torch.matmul(session_output,self.item_embedding.weight[1:].t())
        return result
    
    def predict_top_k(self, session, k=20):
        mask = (session!=0).float()
        length = torch.sum(mask,1).unsqueeze(1).expand((session.shape[0],self.hidden_size))
        mask = mask.unsqueeze(2).expand((session.shape[0],session.shape[1],self.hidden_size))
        session_item_embeddings = self.item_embedding(session) * mask
        positions = session.shape[1] - torch.arange(0,session.shape[1]).unsqueeze(0).expand_as(session).to(device)
        session_position_embeddings = self.position_embedding(positions)*mask
        session_item_vecs = torch.cat((session_item_embeddings,session_position_embeddings), dim=2)
        attention_mask = (session == self.padding_idx)
        sa_output, sa_weights = self.self_attention(session_item_vecs, attention_mask,is_train=False)
        sa_output = self.layer_norm1(sa_output)

        session_position_weights = self.position_weights(positions)*mask
        sa_weights = sa_weights.unsqueeze(2).expand_as(session_item_embeddings)
        session_item_vecs2 = session_item_embeddings * session_position_weights * sa_weights
        psa_output = torch.sum(session_item_vecs2, dim=1)/length
        psa_output = self.layer_norm2(psa_output)

        gen_output =self.activate(self.gen_mlp(sa_output))

        cur_output = self.activate(self.cur_mlp(session_item_embeddings[:,-1]))
        deep_output = self.activate(self.deep_mlp(torch.cat((sa_output,session_item_embeddings[:,-1]),1)))
        session_output = self.layer_norm2(gen_output* cur_output+deep_output +psa_output)

        result = torch.matmul(session_output,self.item_embedding.weight[1:].t())
        result = torch.topk(result,k,dim=1)[1]
        
        return result
        

In [ ]:
epochs=50
def train(args):
    hidden_size = args["hidden_size"] if "hidden_size" in args.keys() else 100
    dropout = args["dropout"] if "dropout" in args.keys()  else 0.5
    attention_method = args["method"] if "method" in args.keys()  else "general"
    lr = args["lr"] if "lr" in args.keys()  else 5e-4
    weight_decay = args["weight_decay"] if "weight_decay" in args.keys()  else 1e-5
    amsgrad = args["amsgrad"] if "amsgrad" in args.keys() else True
    session_length = args["session_length"] if "session_length" in args.keys() else 20
    model = POEM(hidden_size=hidden_size, itemNum=dataset.index_count+1, posNum=session_length+1, padding_idx=0, dropout=dropout,
                 activate="selu",attention_method=attention_method).to(device)
    opti = torch.optim.Adam(model.parameters(),lr=lr,weight_decay=weight_decay,amsgrad=amsgrad)
    patience = args["patience"] if "patience" in args.keys() else 5
    best_model_hr = 0.0
    best_model_mrr = 0.0
    best_r1m = 0.0
    best_model = None
    predict_nums = [1,5,10,20]
    no_improvement_epoch = 0
    for epoch in range(epochs):
        batch_losses = []
        epoch_losses = []
        for i,batch_data in enumerate(dataset.get_batch(batch_size,session_length,phase="train")):
            sessions = torch.tensor(batch_data[0]).to(device)
            target_items = torch.tensor(batch_data[1]).squeeze().to(device)-1
            result_pos = model(sessions)
            loss = loss_function(result_pos,target_items)
            opti.zero_grad()
            loss.backward()
            norm = torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=110)
            opti.step()
            batch_losses.append(loss.cpu().detach().numpy())
            epoch_losses.append(loss.cpu().detach().numpy())
            if i % plot_num == 0:
                time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print("[%s] [%d/%d] %d mean_batch_loss : %0.6f" % (time, epoch+1, epochs, i, np.mean(batch_losses)))
                batch_losses = []
        with torch.no_grad():
            start_test_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print("Start predicting",start_test_time)
            rrs = [0 for _ in range(len(predict_nums))]
            hit_nums = [0 for _ in range(len(predict_nums))]
            ndcgs = [0 for _ in range(len(predict_nums))]
            for i,batch_data in enumerate(dataset.get_batch(batch_size,session_length,phase="test")):
                
                sessions = torch.tensor(batch_data[0]).to(device)
                target_items = np.array(batch_data[1])-1
                y_pred = model.predict_top_k(sessions,20).cpu().numpy()
                
                for j,predict_num in enumerate(predict_nums):
                    hit_nums[j]+=get_hit_num(y_pred[:,:predict_num],target_items)
                    rrs[j]+=get_rr(y_pred[:,:predict_num],target_items)
                    ndcgs[j]+=get_ndcg(y_pred[:,:predict_num],target_items)
                    
            end_test_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            hrs = [hit_num/len(dataset.all_testing_data) for hit_num in hit_nums]
            mrrs = [rr/len(dataset.all_testing_data) for rr in rrs]
            mndcgs = [ndcg/len(dataset.all_testing_data) for ndcg in ndcgs]
            if hrs[-1] + mrrs[-1] > best_r1m:
                print("change best")
                best_model = deepcopy(model)
                best_model_hr = hrs[-1]
                best_model_mrr = mrrs[-1]
                best_r1m = hrs[-1] + mrrs[-1]
                no_improvement_epoch = 0
            else:
                no_improvement_epoch +=1
            print("testing finish [%s] "%end_test_time)
            for k,predict_num in enumerate(predict_nums):
                print("\tHR@%d=%.5f  MRR@%d=%.5f  NDCG@%d=%.5f"%(predict_num,hrs[k],predict_num,mrrs[k],predict_num,mndcgs[k]))
        if no_improvement_epoch>=patience:
            print("early stopping")
            break
    return best_model,best_model_hr,best_model_mrr

# CIKM-Session >= 2
    HR@20=0.65834  MRR@20=0.31324, hyper-parameters: session_length-20, hidden_size-100, lr-0.0005 , amsgrad-True, method-general, dropout-0.5, weight_decay-0.000000. 
        HR@1=0.19843  MRR@1=0.19843  NDCG@1=0.19843
        HR@5=0.44970  MRR@5=0.29180  NDCG@5=0.33116
        HR@10=0.55705  MRR@10=0.30616  NDCG@10=0.36591
        HR@20=0.65834  MRR@20=0.31324  NDCG@20=0.39157
# RR-Session >= 2
    HR@20=0.62795  MRR@20=0.36291, hyper-parameters: session_length-20, hidden_size-100, lr-0.0005 , amsgrad-True, method-general, dropout-0.5, weight_decay-0.000000. 
        HR@1=0.27038  MRR@1=0.27038  NDCG@1=0.27038
        HR@5=0.47748  MRR@5=0.34741  NDCG@5=0.37984
        HR@10=0.55384  MRR@10=0.35770  NDCG@10=0.40462
        HR@20=0.62795  MRR@20=0.36291  NDCG@20=0.42345
# RSC64-Session >= 2
    HR@20=0.71886  MRR@20=0.31580， hyper-parameters: session_length-20, hidden_size-100, lr-0.0005 , amsgrad-True, method-general, dropout-0.3, weight_decay-0.000000. 
        HR@1=0.18341  MRR@1=0.18341  NDCG@1=0.18341
        HR@5=0.48034  MRR@5=0.29042  NDCG@5=0.33760
        HR@10=0.61417  MRR@10=0.30842  NDCG@10=0.38101
        HR@20=0.71886  MRR@20=0.31580  NDCG@20=0.40762
# RSC4-Session >= 2
    HR@20=0.72482  MRR@20=0.31616， hyper-parameters: session_length-20, hidden_size-100, lr-0.0003 , amsgrad-True, method-general, dropout-0.3, weight_decay-0.000000. 
        HR@1=0.18267  MRR@1=0.18267  NDCG@1=0.18267
        HR@5=0.48286  MRR@5=0.29065  NDCG@5=0.33837
        HR@10=0.61471  MRR@10=0.30837  NDCG@10=0.38114
        HR@20=0.72482  MRR@20=0.31616  NDCG@20=0.40915

In [ ]:
hidden_sizes = [100]
dropouts = [0.3,0.5]
attention_methods = ["general"]
lrs = [3e-4,5e-4,1e-3]
session_lengths = [20]
weight_decays = [0]
patience = 10
amsgrads = [True]
best_params = ""
best_all_model = 0.0
best_all_hr = 0.0
best_all_mrr = 0.0
best_all_r1m = 0.0
for session_length in session_lengths:
    for hidden_size in hidden_sizes:
        for amsgrad in amsgrads:
            for attention_method in attention_methods:
                for dropout in dropouts:
                    for weight_decay in weight_decays:
                        for lr in lrs:
                            args = {}
                            print("current model hyper-parameters: session_length=%d, hidden_size=%d, lr=%.4f, amsgrad=%s, method=%s, dropout=%.1f, weight_decay=%.6f. \n" % (session_length,hidden_size,lr,str(amsgrad),attention_method,dropout,weight_decay))
                            args["session_length"] = session_length
                            args["hidden_size"] = hidden_size
                            args["amsgrad"] = amsgrad
                            args["method"] = attention_method
                            args["dropout"] = dropout
                            args["weight_decay"] = weight_decay
                            args["lr"] = lr
                            args["patience"] = patience
                            best_model,best_model_hr,best_model_mrr = train(args)
                            if best_model_hr + best_model_mrr > best_all_r1m:
                                print("best model change")
                                best_all_r1m = best_model_hr + best_model_mrr
                                best_all_hr = best_model_hr
                                best_all_mrr = best_model_mrr
                                best_all_model = best_model
                                best_params = "session_length-%d, hidden_size-%d, lr-%.4f , amsgrad-%s, method-%s, dropout-%.1f, weight_decay-%.6f"%(session_length,hidden_size,lr,str(amsgrad),attention_method,dropout,weight_decay)
                            best_model = None
                            print("current model hyper-parameters: session_length=%d, hidden_size=%d, lr=%.4f, amsgrad=%s, method=%s, dropout=%.1f, weight_decay=%.6f. \n" % (session_length,hidden_size,lr,str(amsgrad),attention_method,dropout,weight_decay))
                            print("current model HR@20=%.5f  MRR@20=%.5f."%(best_model_hr,best_model_mrr))
                            print("the best result so far. HR@20=%.5f  MRR@20=%.5f， hyper-parameters: %s. \n"%(best_all_hr,best_all_mrr,best_params))
print("The best result HR@20=%.5f  MRR@20=%.5f, hyper-parameters: %s. "%(best_all_hr,best_all_mrr,best_params))
print("over.")

current model hyper-parameters: session_length=20, hidden_size=100, lr=0.0003, amsgrad=True, method=general, dropout=0.3, weight_decay=0.000000. 

Start building the all training dataset
The total number of training samples is： (369859, 21)
[2019-12-23 09:53:43] [1/50] 0 mean_batch_loss : 37.880951
Start predicting 2019-12-23 09:53:56
The total number of testing samples is： (55898, 21)
change best
testing finish [2019-12-23 09:53:59] 
	HR@1=0.02136  MRR@1=0.02136  NDCG@1=0.02136
	HR@5=0.07072  MRR@5=0.03898  NDCG@5=0.04682
	HR@10=0.10507  MRR@10=0.04343  NDCG@10=0.05779
	HR@20=0.15761  MRR@20=0.04704  NDCG@20=0.07105
[2019-12-23 09:53:59] [2/50] 0 mean_batch_loss : 9.409584
Start predicting 2019-12-23 09:54:12
change best
testing finish [2019-12-23 09:54:16] 
	HR@1=0.08197  MRR@1=0.08197  NDCG@1=0.08197
	HR@5=0.15984  MRR@5=0.11016  NDCG@5=0.12250
	HR@10=0.19838  MRR@10=0.11525  NDCG@10=0.13491
	HR@20=0.24351  MRR@20=0.11834  NDCG@20=0.14626
[2019-12-23 09:54:16] [3/50] 0 mean_batch_lo

Start predicting 2019-12-23 10:00:32
change best
testing finish [2019-12-23 10:00:36] 
	HR@1=0.16378  MRR@1=0.16378  NDCG@1=0.16378
	HR@5=0.47191  MRR@5=0.27705  NDCG@5=0.32557
	HR@10=0.60322  MRR@10=0.29470  NDCG@10=0.36815
	HR@20=0.70702  MRR@20=0.30206  NDCG@20=0.39459
[2019-12-23 10:00:36] [26/50] 0 mean_batch_loss : 4.118276
Start predicting 2019-12-23 10:00:48
change best
testing finish [2019-12-23 10:00:52] 
	HR@1=0.16366  MRR@1=0.16366  NDCG@1=0.16366
	HR@5=0.47114  MRR@5=0.27646  NDCG@5=0.32492
	HR@10=0.60446  MRR@10=0.29439  NDCG@10=0.36817
	HR@20=0.70747  MRR@20=0.30170  NDCG@20=0.39441
[2019-12-23 10:00:52] [27/50] 0 mean_batch_loss : 4.108572
Start predicting 2019-12-23 10:01:05
change best
testing finish [2019-12-23 10:01:08] 
	HR@1=0.16262  MRR@1=0.16262  NDCG@1=0.16262
	HR@5=0.47200  MRR@5=0.27649  NDCG@5=0.32517
	HR@10=0.60478  MRR@10=0.29432  NDCG@10=0.36821
	HR@20=0.70895  MRR@20=0.30170  NDCG@20=0.39475
[2019-12-23 10:01:09] [28/50] 0 mean_batch_loss : 4.116536
Star

Start predicting 2019-12-23 10:05:32
change best
testing finish [2019-12-23 10:05:35] 
	HR@1=0.16967  MRR@1=0.16967  NDCG@1=0.16967
	HR@5=0.48025  MRR@5=0.28289  NDCG@5=0.33197
	HR@10=0.61364  MRR@10=0.30081  NDCG@10=0.37522
	HR@20=0.71838  MRR@20=0.30824  NDCG@20=0.40191
best model change
current model hyper-parameters: session_length=20, hidden_size=100, lr=0.0003, amsgrad=True, method=general, dropout=0.3, weight_decay=0.000000. 

current model HR@20=0.71838  MRR@20=0.30824.
the best result so far. HR@20=0.71838  MRR@20=0.30824， hyper-parameters: session_length-20, hidden_size-100, lr-0.0003 , amsgrad-True, method-general, dropout-0.3, weight_decay-0.000000. 

current model hyper-parameters: session_length=20, hidden_size=100, lr=0.0005, amsgrad=True, method=general, dropout=0.3, weight_decay=0.000000. 

[2019-12-23 10:05:35] [1/50] 0 mean_batch_loss : 38.193760
Start predicting 2019-12-23 10:05:42
change best
testing finish [2019-12-23 10:05:45] 
	HR@1=0.03387  MRR@1=0.03387  NDCG@

change best
testing finish [2019-12-23 10:09:24] 
	HR@1=0.17289  MRR@1=0.17289  NDCG@1=0.17289
	HR@5=0.47835  MRR@5=0.28369  NDCG@5=0.33208
	HR@10=0.61004  MRR@10=0.30143  NDCG@10=0.37482
	HR@20=0.71591  MRR@20=0.30893  NDCG@20=0.40179
[2019-12-23 10:09:24] [24/50] 0 mean_batch_loss : 3.955426
Start predicting 2019-12-23 10:09:31
change best
testing finish [2019-12-23 10:09:34] 
	HR@1=0.17239  MRR@1=0.17239  NDCG@1=0.17239
	HR@5=0.47887  MRR@5=0.28367  NDCG@5=0.33220
	HR@10=0.61110  MRR@10=0.30145  NDCG@10=0.37509
	HR@20=0.71664  MRR@20=0.30894  NDCG@20=0.40199
[2019-12-23 10:09:34] [25/50] 0 mean_batch_loss : 3.955344
Start predicting 2019-12-23 10:09:41
change best
testing finish [2019-12-23 10:09:44] 
	HR@1=0.17496  MRR@1=0.17496  NDCG@1=0.17496
	HR@5=0.47751  MRR@5=0.28485  NDCG@5=0.33276
	HR@10=0.61229  MRR@10=0.30305  NDCG@10=0.37655
	HR@20=0.71795  MRR@20=0.31052  NDCG@20=0.40344
[2019-12-23 10:09:44] [26/50] 0 mean_batch_loss : 3.809392
Start predicting 2019-12-23 10:09:51
test

Start predicting 2019-12-23 10:13:41
testing finish [2019-12-23 10:13:44] 
	HR@1=0.18457  MRR@1=0.18457  NDCG@1=0.18457
	HR@5=0.47876  MRR@5=0.29028  NDCG@5=0.33708
	HR@10=0.61224  MRR@10=0.30820  NDCG@10=0.38035
	HR@20=0.71754  MRR@20=0.31562  NDCG@20=0.40712
[2019-12-23 10:13:44] [50/50] 0 mean_batch_loss : 3.675073
Start predicting 2019-12-23 10:13:51
testing finish [2019-12-23 10:13:54] 
	HR@1=0.18471  MRR@1=0.18471  NDCG@1=0.18471
	HR@5=0.47910  MRR@5=0.29098  NDCG@5=0.33772
	HR@10=0.61240  MRR@10=0.30895  NDCG@10=0.38100
	HR@20=0.71774  MRR@20=0.31639  NDCG@20=0.40779
best model change
current model hyper-parameters: session_length=20, hidden_size=100, lr=0.0005, amsgrad=True, method=general, dropout=0.3, weight_decay=0.000000. 

current model HR@20=0.71886  MRR@20=0.31580.
the best result so far. HR@20=0.71886  MRR@20=0.31580， hyper-parameters: session_length-20, hidden_size-100, lr-0.0005 , amsgrad-True, method-general, dropout-0.3, weight_decay-0.000000. 

current model hyper-

Start predicting 2019-12-23 10:18:19
testing finish [2019-12-23 10:18:23] 
	HR@1=0.17929  MRR@1=0.17929  NDCG@1=0.17929
	HR@5=0.47594  MRR@5=0.28605  NDCG@5=0.33320
	HR@10=0.60773  MRR@10=0.30376  NDCG@10=0.37594
	HR@20=0.71439  MRR@20=0.31130  NDCG@20=0.40309
[2019-12-23 10:18:23] [24/50] 0 mean_batch_loss : 3.574958
Start predicting 2019-12-23 10:18:37
testing finish [2019-12-23 10:18:41] 
	HR@1=0.17972  MRR@1=0.17972  NDCG@1=0.17972
	HR@5=0.47581  MRR@5=0.28667  NDCG@5=0.33365
	HR@10=0.60770  MRR@10=0.30439  NDCG@10=0.37642
	HR@20=0.71373  MRR@20=0.31187  NDCG@20=0.40338
[2019-12-23 10:18:41] [25/50] 0 mean_batch_loss : 3.620137
Start predicting 2019-12-23 10:18:55
testing finish [2019-12-23 10:18:59] 
	HR@1=0.18144  MRR@1=0.18144  NDCG@1=0.18144
	HR@5=0.47687  MRR@5=0.28799  NDCG@5=0.33490
	HR@10=0.60732  MRR@10=0.30550  NDCG@10=0.37719
	HR@20=0.71405  MRR@20=0.31300  NDCG@20=0.40429
[2019-12-23 10:18:59] [26/50] 0 mean_batch_loss : 3.619580
Start predicting 2019-12-23 10:19:13
tes

Start predicting 2019-12-23 10:25:04
change best
testing finish [2019-12-23 10:25:08] 
	HR@1=0.15906  MRR@1=0.15906  NDCG@1=0.15906
	HR@5=0.44932  MRR@5=0.26625  NDCG@5=0.31184
	HR@10=0.57521  MRR@10=0.28314  NDCG@10=0.35264
	HR@20=0.67550  MRR@20=0.29024  NDCG@20=0.37817
[2019-12-23 10:25:08] [17/50] 0 mean_batch_loss : 4.976556
Start predicting 2019-12-23 10:25:21
change best
testing finish [2019-12-23 10:25:24] 
	HR@1=0.16061  MRR@1=0.16061  NDCG@1=0.16061
	HR@5=0.45311  MRR@5=0.26841  NDCG@5=0.31441
	HR@10=0.57990  MRR@10=0.28542  NDCG@10=0.35549
	HR@20=0.68053  MRR@20=0.29254  NDCG@20=0.38111
[2019-12-23 10:25:24] [18/50] 0 mean_batch_loss : 4.703499
Start predicting 2019-12-23 10:25:39
change best
testing finish [2019-12-23 10:25:43] 
	HR@1=0.15940  MRR@1=0.15940  NDCG@1=0.15940
	HR@5=0.45590  MRR@5=0.26900  NDCG@5=0.31556
	HR@10=0.58313  MRR@10=0.28607  NDCG@10=0.35680
	HR@20=0.68425  MRR@20=0.29321  NDCG@20=0.38252
[2019-12-23 10:25:43] [19/50] 0 mean_batch_loss : 4.602868
Star

Start predicting 2019-12-23 10:34:25
testing finish [2019-12-23 10:34:29] 
	HR@1=0.16414  MRR@1=0.16414  NDCG@1=0.16414
	HR@5=0.47297  MRR@5=0.27743  NDCG@5=0.32609
	HR@10=0.60897  MRR@10=0.29568  NDCG@10=0.37018
	HR@20=0.71294  MRR@20=0.30303  NDCG@20=0.39664
[2019-12-23 10:34:29] [42/50] 0 mean_batch_loss : 3.965297
Start predicting 2019-12-23 10:34:49
change best
testing finish [2019-12-23 10:34:53] 
	HR@1=0.16544  MRR@1=0.16544  NDCG@1=0.16544
	HR@5=0.47433  MRR@5=0.27858  NDCG@5=0.32730
	HR@10=0.60931  MRR@10=0.29673  NDCG@10=0.37107
	HR@20=0.71262  MRR@20=0.30405  NDCG@20=0.39739
[2019-12-23 10:34:54] [43/50] 0 mean_batch_loss : 4.018471
Start predicting 2019-12-23 10:35:13
change best
testing finish [2019-12-23 10:35:17] 
	HR@1=0.16419  MRR@1=0.16419  NDCG@1=0.16419
	HR@5=0.47572  MRR@5=0.27825  NDCG@5=0.32738
	HR@10=0.60915  MRR@10=0.29620  NDCG@10=0.37068
	HR@20=0.71366  MRR@20=0.30363  NDCG@20=0.39731
[2019-12-23 10:35:17] [44/50] 0 mean_batch_loss : 3.882136
Start predicting

Start predicting 2019-12-23 10:44:06
change best
testing finish [2019-12-23 10:44:10] 
	HR@1=0.16024  MRR@1=0.16024  NDCG@1=0.16024
	HR@5=0.46771  MRR@5=0.27377  NDCG@5=0.32207
	HR@10=0.59907  MRR@10=0.29146  NDCG@10=0.36471
	HR@20=0.70246  MRR@20=0.29877  NDCG@20=0.39102
[2019-12-23 10:44:10] [16/50] 0 mean_batch_loss : 4.470697
Start predicting 2019-12-23 10:44:30
change best
testing finish [2019-12-23 10:44:34] 
	HR@1=0.16487  MRR@1=0.16487  NDCG@1=0.16487
	HR@5=0.46755  MRR@5=0.27606  NDCG@5=0.32373
	HR@10=0.60104  MRR@10=0.29405  NDCG@10=0.36707
	HR@20=0.70457  MRR@20=0.30138  NDCG@20=0.39342
[2019-12-23 10:44:34] [17/50] 0 mean_batch_loss : 4.136858
Start predicting 2019-12-23 10:44:54
change best
testing finish [2019-12-23 10:44:58] 
	HR@1=0.16369  MRR@1=0.16369  NDCG@1=0.16369
	HR@5=0.47014  MRR@5=0.27584  NDCG@5=0.32418
	HR@10=0.60340  MRR@10=0.29380  NDCG@10=0.36745
	HR@20=0.70666  MRR@20=0.30110  NDCG@20=0.39373
[2019-12-23 10:44:58] [18/50] 0 mean_batch_loss : 4.292415
Star

Start predicting 2019-12-23 10:54:09
testing finish [2019-12-23 10:54:13] 
	HR@1=0.16403  MRR@1=0.16403  NDCG@1=0.16403
	HR@5=0.47560  MRR@5=0.27846  NDCG@5=0.32753
	HR@10=0.61201  MRR@10=0.29686  NDCG@10=0.37184
	HR@20=0.71969  MRR@20=0.30447  NDCG@20=0.39924
[2019-12-23 10:54:13] [41/50] 0 mean_batch_loss : 3.637409
Start predicting 2019-12-23 10:54:32
change best
testing finish [2019-12-23 10:54:36] 
	HR@1=0.16582  MRR@1=0.16582  NDCG@1=0.16582
	HR@5=0.47769  MRR@5=0.27978  NDCG@5=0.32903
	HR@10=0.61401  MRR@10=0.29814  NDCG@10=0.37328
	HR@20=0.72090  MRR@20=0.30568  NDCG@20=0.40045
[2019-12-23 10:54:37] [42/50] 0 mean_batch_loss : 4.011255
Start predicting 2019-12-23 10:54:57
testing finish [2019-12-23 10:55:01] 
	HR@1=0.16315  MRR@1=0.16315  NDCG@1=0.16315
	HR@5=0.47730  MRR@5=0.27847  NDCG@5=0.32796
	HR@10=0.61364  MRR@10=0.29683  NDCG@10=0.37221
	HR@20=0.71911  MRR@20=0.30431  NDCG@20=0.39908
[2019-12-23 10:55:01] [43/50] 0 mean_batch_loss : 3.923194
Start predicting 2019-12-23 